In [1]:
import os, sys, numpy as np, torch, torch.nn as nn, pandas as pd, matplotlib.pyplot as plt

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CURRENT_DIR = os.getcwd()
sys.path.append(f"{CURRENT_DIR}/../..")
from utils import ScorerStepByStep


# ---------------- MODEL ---------------- #
class PredictionModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.input_dim = 64        # raw32 + rm5_32
        self.output_dim = 32       # next raw32
        self.hidden_size = 64
        self.num_layers = 1

        self.lstm = nn.LSTM(self.input_dim, self.hidden_size, self.num_layers, batch_first=True)
        self.fc = nn.Linear(self.hidden_size, self.output_dim)

        weights_path = os.path.join(CURRENT_DIR, "lstm_weights.pt")
        print("Loading:", weights_path)
        self.load_state_dict(torch.load(weights_path, map_location=DEVICE))
        self.to(DEVICE)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1])


# -------------- MAKE SEQUENCES -------------- #
def make_sequences(arr64, seq_len=64):
    X, y = [], []
    for i in range(len(arr64) - seq_len):
        X.append(arr64[i:i+seq_len])
        y.append(arr64[i+seq_len, :32])   # next raw32 target
    return np.stack(X), np.stack(y)


# -------------- LOAD DATA -------------- #
seq_id = 516    # change this to visualize any sequence
train_file = f"{CURRENT_DIR}/../../datasets/train.parquet"
scorer = ScorerStepByStep(train_file)
df = scorer.dataset

df_seq = df[df["seq_ix"] == seq_id].sort_values("step_in_seq").reset_index(drop=True)

# raw32
raw = df_seq.iloc[:, 3:35].to_numpy().astype(np.float32)
# rolling mean 5 → rm5
rm5 = pd.DataFrame(raw).rolling(5, min_periods=1).mean().to_numpy().astype(np.float32)
# concatenate → 64-dim feature
arr64 = np.concatenate([raw, rm5], axis=1).astype(np.float32)

X_np, y_true = make_sequences(arr64, seq_len=64)
X = torch.tensor(X_np, dtype=torch.float32).to(DEVICE)

# -------------- PREDICT -------------- #
model = PredictionModel()
model.eval()
with torch.no_grad():
    y_pred = model(X).cpu().numpy()

print("Shapes:", y_true.shape, y_pred.shape)


# -------------- PLOT -------------- #
for k in range(32):
    plt.figure(figsize=(20, 4))
    plt.plot(y_true[:, k], label="True", linewidth=1)
    plt.plot(y_pred[:, k], label="Predicted", linewidth=1)
    plt.title(f"seq_ix={seq_id} — Feature {k} (True vs Predicted)")
    plt.xlabel("Time index (after seq-length offset)")
    plt.ylabel("Z-normalized feature value")
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

print("\nDone plotting ✓")


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading: /Users/christiankwok/my wunderfund_challenge/wunderfund_challenge/competition_package/examples/delta_predict_2/lstm_weights.pt


FileNotFoundError: [Errno 2] No such file or directory: '/Users/christiankwok/my wunderfund_challenge/wunderfund_challenge/competition_package/examples/delta_predict_2/lstm_weights.pt'